In [2]:
import sentencepiece as spm
import pandas as pd
import numpy as np
import time

from sklearn.feature_extraction.text import *
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [3]:
df_path = '../../main_dataset/'
trained_model_and_fitted_encoder_path = '../models/'
plots_path = '../plots/'

url = 'https://drive.google.com/file/d/1JAPtGXXHC0f5tKgHuS1b1ErefBXoYZEl/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id=' + url.split('/')[-2]
df = pd.read_csv(path)

In [4]:
df.shape

(76466, 3)

In [8]:
df['type_text'].unique()

array(['Гомдол', 'Санал хүсэлт', 'Асуулт', 'Талархал', 'Шүүмжлэл',
       'Өргөдөл'], dtype=object)

In [16]:
sp = spm.SentencePieceProcessor()
sp.Load('../mongolian_bert_sentencepiece/mn_uncased.model')

def sp_tokenize(w):
    return sp.EncodeAsPieces(w)

In [17]:
import re

# Only removing numbers and hyphens
#df['content'] = [re.sub('\d', '', sent) for sent in df['content']]
#df['content'] = [re.sub('-', ' ', sent) for sent in df['content']]
#df['content'] = [re.sub(' \s+', ' ', sent) for sent in df['content']]

In [18]:
# Stratified train and test split
train, test = train_test_split(df, test_size=0.1, random_state=999, stratify=df['type_text'])

text_clf = Pipeline([('vect', CountVectorizer(tokenizer=sp_tokenize, lowercase=True)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-4, max_iter=5, random_state=0))])

t = time.time()
text_clf = text_clf.fit(train['content'], train['type_text'])
t = time.time()-t
print("Training time in seconds: ", t)

t = time.time()
predicted = text_clf.predict(test['content'])
t = time.time()-t
print("Prediction time in seconds: ", t)

print("Feature count:", len(text_clf.named_steps['vect'].vocabulary_))

acc = np.mean(predicted == test['type_text'])
print("Classifier accuracy: ", acc)

Training time in seconds:  8.341360807418823
Prediction time in seconds:  0.8103790283203125
Feature count: 25824
Classifier accuracy:  0.7876291356087355


In [6]:
to_pred = [
    'Би шинэ апп татаад орох гэхээр орохгүй шууд гарчихаад',
    'Танай сүлжээ болохгүй байх шиг байна хэзээ хэвийн болох вэ',
    'одоо тэр мөнгөө яаж авч байж хаах уу',
    'Е-код интернэтээр худалдан авалтын код онлайнаар авч болох уу'
]

for entry in to_pred:
    print('Text:', entry)
    print('Pred:', text_clf.predict([entry])[0], '\n')

Text: Би шинэ апп татаад орох гэхээр орохгүй шууд гарчихаад
Pred: Санал хүсэлт 

Text: Танай сүлжээ болохгүй байх шиг байна хэзээ хэвийн болох вэ
Pred: Санал хүсэлт 

Text: одоо тэр мөнгөө яаж авч байж хаах уу
Pred: Санал хүсэлт 

Text: Е-код интернэтээр худалдан авалтын код онлайнаар авч болох уу
Pred: Санал хүсэлт 

